In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import random
from datetime import date
import random
import plotly.plotly as py
import plotly.graph_objs as go
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE

In [2]:
#Connect to database
engine = create_engine('postgresql://earlynreinhardt@localhost:5432/athena_database')
austin_db = pd.read_sql('atx_patient_db', engine)

In [3]:
office_entry_db = pd.DataFrame(columns = ['patient_id', 'doctor_id', 'date_of_visit', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5'])

In [4]:
#Create office_entry and office_exit dataframes
#TODO: Add Neutral as a Score
# -1 - Negative
# 0 - Neutral
# 1 - Postive 
patient_id = np.random.randint(low=1, high=601, size=600)

office_entry_responses = {'patient_id': patient_id, 
    'Q1': np.random.choice(2, size=600, p=[0.35, 0.65]), 
    'Q2': np.random.choice(2, size=600, p=[0.40, 0.60]), 
    'Q3': np.random.choice(2, size=600, p=[0.70, 0.30]), 
    'Q4': np.random.choice(2, size=600, p=[0.10, 0.90]), 
    'Q5': np.random.choice(2, size=600, p=[0.55, 0.45])}

office_exit_responses = {'patient_id': patient_id, 
    'Q1': np.random.choice(2, size=600, p=[0.30, 0.70]), 
    'Q2': np.random.choice(2, size=600, p=[0.60, 0.40]), 
    'Q3': np.random.choice(2, size=600, p=[0.50, 0.50]), 
    'Q4': np.random.choice(2, size=600, p=[0.05, 0.95]), 
    'Q5': np.random.choice(2, size=600, p=[0.45, 0.55])}


office_entry_db = pd.DataFrame.from_dict(office_entry_responses)
office_exit_db = pd.DataFrame.from_dict(office_exit_responses)

In [5]:
office_entry_db['average'] = office_entry_db[['Q1', 'Q2', 'Q3', 'Q4', 'Q5']].mean(axis=1)
office_exit_db['average'] = office_exit_db[['Q1', 'Q2', 'Q3', 'Q4', 'Q5']].mean(axis=1)

In [6]:
office_entry_db['overall_score'] = office_entry_db.apply(lambda row: 1 if row['average'] >= 0.5 else 0, axis=1)
office_exit_db['overall_score'] = office_exit_db.apply(lambda row: 1 if row['average'] >= 0.5 else 0, axis=1)

In [7]:
def assign_doctor(row):
    row = random.randint(1,5)
    return row

In [8]:
office_entry_db['doctor'] = 1
office_entry_db['doctor'] = office_entry_db.doctor.apply(assign_doctor)
office_exit_db['doctor'] = office_entry_db['doctor']

In [9]:
office_exit_db.head()

,Q1,Q2,Q3,Q4,Q5,patient_id,average,overall_score,doctor
0,1,0,1,1,0,243,0.6,1,1
1,1,0,0,1,0,107,0.4,0,2
2,0,0,1,1,1,5,0.6,1,5
3,0,0,1,1,1,145,0.6,1,1
4,0,0,0,1,1,556,0.4,0,1


In [10]:
office_entry_db.head()

,Q1,Q2,Q3,Q4,Q5,patient_id,average,overall_score,doctor
0,1,0,0,1,1,243,0.6,1,1
1,0,1,1,0,0,107,0.4,0,2
2,1,1,0,1,1,5,0.8,1,5
3,0,1,1,1,0,145,0.6,1,1
4,0,0,0,1,0,556,0.2,0,1


In [11]:
def get_visit_date(row): 
    start_date = date.today().replace(day=1, month=1).toordinal()
    end_date = date.today().toordinal()
    random_day = date.fromordinal(random.randint(start_date, end_date))
    row = random_day
    return row

In [12]:
office_entry_db['visit_date'] = 1
office_exit_db['visit_date'] = 1

In [13]:
office_exit_db

,Q1,Q2,Q3,Q4,Q5,patient_id,average,overall_score,doctor,visit_date
0,1,0,1,1,0,243,0.6,1,1,1
1,1,0,0,1,0,107,0.4,0,2,1
2,0,0,1,1,1,5,0.6,1,5,1
3,0,0,1,1,1,145,0.6,1,1,1
4,0,0,0,1,1,556,0.4,0,1,1
5,1,0,0,0,0,578,0.2,0,2,1
6,1,0,0,1,1,189,0.6,1,2,1
7,1,1,1,1,0,577,0.8,1,3,1
8,1,1,1,1,0,3,0.8,1,1,1
9,1,0,1,1,1,286,0.8,1,4,1


In [14]:
office_entry_db

,Q1,Q2,Q3,Q4,Q5,patient_id,average,overall_score,doctor,visit_date
0,1,0,0,1,1,243,0.6,1,1,1
1,0,1,1,0,0,107,0.4,0,2,1
2,1,1,0,1,1,5,0.8,1,5,1
3,0,1,1,1,0,145,0.6,1,1,1
4,0,0,0,1,0,556,0.2,0,1,1
5,1,1,0,1,1,578,0.8,1,2,1
6,1,0,1,1,0,189,0.6,1,2,1
7,1,1,0,1,0,577,0.6,1,3,1
8,0,0,1,1,1,3,0.6,1,1,1
9,1,1,0,1,1,286,0.8,1,4,1


In [15]:
office_entry_db['visit_date'] = office_entry_db.applymap(get_visit_date)
office_exit_db['visit_date'] = office_exit_db.applymap(get_visit_date)

In [16]:
#Visualization 
office_exit_db

,Q1,Q2,Q3,Q4,Q5,patient_id,average,overall_score,doctor,visit_date
0,1,0,1,1,0,243,0.6,1,1,2017-02-07
1,1,0,0,1,0,107,0.4,0,2,2017-01-26
2,0,0,1,1,1,5,0.6,1,5,2017-01-18
3,0,0,1,1,1,145,0.6,1,1,2017-04-04
4,0,0,0,1,1,556,0.4,0,1,2017-04-06
5,1,0,0,0,0,578,0.2,0,2,2017-04-05
6,1,0,0,1,1,189,0.6,1,2,2017-02-16
7,1,1,1,1,0,577,0.8,1,3,2017-03-18
8,1,1,1,1,0,3,0.8,1,1,2017-01-14
9,1,0,1,1,1,286,0.8,1,4,2017-04-08


In [17]:
office_entry_db['day'] = office_entry_db['visit_date'].apply(lambda x: x.day)
office_entry_db['month'] = office_entry_db['visit_date'].apply(lambda x: x.month)
office_entry_db['year'] = office_entry_db['visit_date'].apply(lambda x: x.year)
office_entry_db['weekday'] = office_entry_db['visit_date'].apply(lambda x: x.weekday())

In [18]:
office_exit_db['day'] = office_exit_db['visit_date'].apply(lambda x: x.day)
office_exit_db['month'] = office_exit_db['visit_date'].apply(lambda x: x.month)
office_exit_db['year'] = office_exit_db['visit_date'].apply(lambda x: x.year)
office_exit_db['weekday'] = office_exit_db['visit_date'].apply(lambda x: x.weekday())

In [19]:
office_entry_db

,Q1,Q2,Q3,Q4,Q5,patient_id,average,overall_score,doctor,visit_date,day,month,year,weekday
0,1,0,0,1,1,243,0.6,1,1,2017-02-04,4,2,2017,5
1,0,1,1,0,0,107,0.4,0,2,2017-04-14,14,4,2017,4
2,1,1,0,1,1,5,0.8,1,5,2017-03-20,20,3,2017,0
3,0,1,1,1,0,145,0.6,1,1,2017-04-26,26,4,2017,2
4,0,0,0,1,0,556,0.2,0,1,2017-01-31,31,1,2017,1
5,1,1,0,1,1,578,0.8,1,2,2017-04-27,27,4,2017,3
6,1,0,1,1,0,189,0.6,1,2,2017-01-12,12,1,2017,3
7,1,1,0,1,0,577,0.6,1,3,2017-01-19,19,1,2017,3
8,0,0,1,1,1,3,0.6,1,1,2017-02-24,24,2,2017,4
9,1,1,0,1,1,286,0.8,1,4,2017-04-08,8,4,2017,5


In [20]:
office_exit_db

,Q1,Q2,Q3,Q4,Q5,patient_id,average,overall_score,doctor,visit_date,day,month,year,weekday
0,1,0,1,1,0,243,0.6,1,1,2017-02-07,7,2,2017,1
1,1,0,0,1,0,107,0.4,0,2,2017-01-26,26,1,2017,3
2,0,0,1,1,1,5,0.6,1,5,2017-01-18,18,1,2017,2
3,0,0,1,1,1,145,0.6,1,1,2017-04-04,4,4,2017,1
4,0,0,0,1,1,556,0.4,0,1,2017-04-06,6,4,2017,3
5,1,0,0,0,0,578,0.2,0,2,2017-04-05,5,4,2017,2
6,1,0,0,1,1,189,0.6,1,2,2017-02-16,16,2,2017,3
7,1,1,1,1,0,577,0.8,1,3,2017-03-18,18,3,2017,5
8,1,1,1,1,0,3,0.8,1,1,2017-01-14,14,1,2017,5
9,1,0,1,1,1,286,0.8,1,4,2017-04-08,8,4,2017,5


In [21]:
#Do a Database Join 
total_office_scores = office_entry_db.join(office_exit_db, on='patient_id', how='left',lsuffix='_entry', rsuffix='_exit',sort=False)

In [22]:
total_office_scores

,Q1_entry,Q2_entry,Q3_entry,Q4_entry,Q5_entry,patient_id_entry,average_entry,overall_score_entry,doctor_entry,visit_date_entry,...,Q5_exit,patient_id_exit,average_exit,overall_score_exit,doctor_exit,visit_date_exit,day_exit,month_exit,year_exit,weekday_exit
0,1,0,0,1,1,243,0.6,1,1,2017-02-04,...,1.0,464.0,0.8,1.0,4.0,2017-02-26,26.0,2.0,2017.0,6.0
1,0,1,1,0,0,107,0.4,0,2,2017-04-14,...,1.0,34.0,1.0,1.0,2.0,2017-04-08,8.0,4.0,2017.0,5.0
2,1,1,0,1,1,5,0.8,1,5,2017-03-20,...,0.0,578.0,0.2,0.0,2.0,2017-04-05,5.0,4.0,2017.0,2.0
3,0,1,1,1,0,145,0.6,1,1,2017-04-26,...,1.0,137.0,0.8,1.0,3.0,2017-02-25,25.0,2.0,2017.0,5.0
4,0,0,0,1,0,556,0.2,0,1,2017-01-31,...,1.0,389.0,0.8,1.0,3.0,2017-01-14,14.0,1.0,2017.0,5.0
5,1,1,0,1,1,578,0.8,1,2,2017-04-27,...,0.0,480.0,0.2,0.0,5.0,2017-02-28,28.0,2.0,2017.0,1.0
6,1,0,1,1,0,189,0.6,1,2,2017-01-12,...,1.0,385.0,0.6,1.0,2.0,2017-02-22,22.0,2.0,2017.0,2.0
7,1,1,0,1,0,577,0.6,1,3,2017-01-19,...,0.0,281.0,0.4,0.0,2.0,2017-01-20,20.0,1.0,2017.0,4.0
8,0,0,1,1,1,3,0.6,1,1,2017-02-24,...,1.0,145.0,0.6,1.0,1.0,2017-04-04,4.0,4.0,2017.0,1.0
9,1,1,0,1,1,286,0.8,1,4,2017-04-08,...,1.0,351.0,0.6,1.0,5.0,2017-02-22,22.0,2.0,2017.0,2.0


In [134]:
#Visualizations

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']
day = range(1, 32)
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

#Office Entry vs Office Exit 

trace1 = go.Bar(
    x=['Bad', 'Good'],
    y=list(office_entry_db.groupby('overall_score').count()['Q1']),
    name='Office Entry'
)
trace2 = go.Bar(
    x=['Bad', 'Good'],
    y=list(office_exit_db.groupby('overall_score').count()['Q1']),
    name='Office Exit'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group', 
    title = 'Patient Mood per Stage', 
    xaxis = dict(title = 'Sentiment'),
    yaxis = dict(title = 'Number of Patients')
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar')

In [96]:
#Simple Confusion Matrix
trace = go.Heatmap(z=[[57, 142], [97, 303]],
                   x=['Bad', 'Good'],
                   y=['Bad', 'Good'])
data=[trace]
py.iplot(data, filename='labelled-heatmap')

In [106]:
list(total_office_scores.groupby(['doctor_entry', 'overall_score_entry', 'overall_score_exit']).count().query('doctor_entry == 1')['Q1_entry'])

[13, 24, 25, 62]

In [132]:
#Doctors Satisfaction Count 

trace0 = go.Bar(
    x=['Bad - Bad', 'Bad - Good', 'Good - Bad', 'Good - Good'],
    y=list(total_office_scores.groupby(['doctor_entry', 'overall_score_entry', 'overall_score_exit']).count().query('doctor_entry == 1')['Q1_entry']),
    name='Doctor One',
    marker=dict(
        color='rgb(49,130,189)'
    )
)
trace1 = go.Bar(
    x=['Bad - Bad', 'Bad - Good', 'Good - Bad', 'Good - Good'],
    y=list(total_office_scores.groupby(['doctor_entry', 'overall_score_entry', 'overall_score_exit']).count().query('doctor_entry == 2')['Q1_entry']),
    name='Doctor Two',
    marker=dict(
        color='rgb(102, 54, 127)',
    )
)

trace2 = go.Bar(
    x=['Bad - Bad', 'Bad - Good', 'Good - Bad', 'Good - Good'],
    y=list(total_office_scores.groupby(['doctor_entry', 'overall_score_entry', 'overall_score_exit']).count().query('doctor_entry == 3')['Q1_entry']),
    name='Doctor Three',
    marker=dict(
        color='rgb(22, 183, 38)',
    )
)


trace3 = go.Bar(
    x=['Bad - Bad', 'Bad - Good', 'Good - Bad', 'Good - Good'],
    y=list(total_office_scores.groupby(['doctor_entry', 'overall_score_entry', 'overall_score_exit']).count().query('doctor_entry == 4')['Q1_entry']),
    name='Doctor Four',
    marker=dict(
        color='rgb(183, 22, 153)',
    )
)


trace4 = go.Bar(
    x=['Bad - Bad', 'Bad - Good', 'Good - Bad', 'Good - Good'],
    y=list(total_office_scores.groupby(['doctor_entry', 'overall_score_entry', 'overall_score_exit']).count().query('doctor_entry == 5')['Q1_entry']),
    name='Doctor Five',
    marker=dict(
        color='rgb(38, 190, 201)',
    )
)


data = [trace0, trace1, trace2, trace3, trace4]
layout = go.Layout(
    title = 'Patient Satisfaction Journey',
    xaxis = dict(tickangle=-45, title = 'Satisfaction Path'),
    yaxis = dict(title = 'Number of Patients Per Score'),
    barmode='group',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='angled-text-bar')


In [118]:
#Office Exit Bar Graph 
total_office_scores.groupby(['month_entry', 'doctor_entry', 'overall_score_entry', 'overall_score_exit']).count().query('doctor_entry == 5')['Q1_entry']



month_entry  doctor_entry  overall_score_entry  overall_score_exit
1            5             0                    0                      2
                                                1                      7
                           1                    0                      5
                                                1                     15
2            5             0                    0                      1
                                                1                      5
                           1                    0                      3
                                                1                     18
3            5             0                    0                      1
                                                1                      9
                           1                    0                     10
                                                1                     15
4            5             0                    0        

In [123]:
doctor_one_scores_by_month = list(total_office_scores.groupby(['month_entry', 'doctor_entry', 'overall_score_entry', 'overall_score_exit']).count().query('doctor_entry == 5')['Q1_entry'])
doctor_one_scores_by_month[3::4]

[15, 18, 15, 13]

In [135]:
#Office Entry Bar Graph By Month

trace0 = go.Scatter(
    x=['Jan', 'Feb', 'Mar', 'Apr'],
    y=doctor_one_scores_by_month[::4],
    name='Bad to Bad',
    marker=dict(
        color='rgb(49,130,189)'
    )
)
trace1 = go.Scatter(
    x=['Jan', 'Feb', 'Mar', 'Apr'],
    y=doctor_one_scores_by_month[1::4],
    name='Bad to Good',
    marker=dict(
        color='rgb(102, 54, 127)',
    )
)

trace2 = go.Scatter(
    x=['Jan', 'Feb', 'Mar', 'Apr'],
    y=doctor_one_scores_by_month[2::4],
    name='Good to Bad',
    marker=dict(
        color='rgb(22, 183, 38)'
    )
)
trace3 = go.Scatter(
    x=['Jan', 'Feb', 'Mar', 'Apr'],
    y=doctor_one_scores_by_month[3::4],
    name='Good to Good',
    marker=dict(
        color='rgb(183, 22, 153)',
    )
)


data = [trace0, trace1,trace2, trace3]
layout = go.Layout(
    title = 'Patients Satisfaction Journey for Doctor One',
    xaxis = dict(tickangle=-45, title= 'Month'),
    yaxis = dict(title='Count of Patient'),
    barmode='group',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Patient_Satisfaction')





In [2]:
#Tweet Sentiment Analysis - Looking at Text 
tweet_data = pd.read_csv('doctor_tweets.csv', names = ['Sentiment', '-', 'Tweet'])

In [3]:
tweet_data.head()

,Sentiment,-,Tweet
0,negative,-,"[""I'm studying to be a doctor with a harsh, ha..."
1,negative,-,['Praise @hubblecontacts for saving my \xf0\x9...
2,negative,-,['RT @PrensaRural: #LaESTAMPIDA por la revocat...
3,negative,-,"['RT @mtonaaguilar: Fervor, tradici\xc3\xb3n y..."
4,negative,-,"[""https://t.co/wcR4fD2Ak5\n\nAfter this week's..."


In [65]:
pos_neg_tweet_data = tweet_data.query('Sentiment == "negative" or Sentiment == "positive"')

In [59]:
entry_pos_neg_tweet_data_subsample = pos_neg_tweet_data.sample(n=599, replace=False, random_state=42)
exit_pos_neg_tweet_data_subsample = pos_neg_tweet_data.sample(n=599, replace=False, random_state=42)


In [66]:
pos_neg_tweet_data['Sentiment_score'] = pos_neg_tweet_data['Sentiment'].map(lambda x: 1 if x == 'positive' else 0)

/Users/earlynreinhardt/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [67]:
#Bag of Words! Simple 
pos_neg_tweet_data

,Sentiment,-,Tweet,Sentiment_score
0,negative,-,"[""I'm studying to be a doctor with a harsh, ha...",0
1,negative,-,['Praise @hubblecontacts for saving my \xf0\x9...,0
2,negative,-,['RT @PrensaRural: #LaESTAMPIDA por la revocat...,0
3,negative,-,"['RT @mtonaaguilar: Fervor, tradici\xc3\xb3n y...",0
4,negative,-,"[""https://t.co/wcR4fD2Ak5\n\nAfter this week's...",0
5,negative,-,"[""Now even Kim Kardashian's doctor says her bu...",0
6,negative,-,['The Doctor || Doctor Who\n\n \xe2\x80\x94 ...,0
7,negative,-,['The doctor and he can do it all oh yeah and ...,0
8,negative,-,"[""RT @colesprouse: When the doctor asks me how...",0
9,negative,-,"[""@peterdaou Does @mtaibbi understand that peo...",0


In [70]:
c_vectorizer = CountVectorizer(decode_error= 'ignore', min_df = 20, ngram_range=(1,2) )
c_X = c_vectorizer.fit_transform(pos_neg_tweet_data.Tweet).toarray()


In [77]:
tSNE = TSNE(n_components=3, random_state=42)

In [78]:
tSNE_tweets = tSNE.fit_transform(c_X)

In [82]:
tSNE_tweets_with_scores = np.column_stack((tSNE_tweets,pos_neg_tweet_data['Sentiment_score'].values))

In [88]:
tsne_df = pd.DataFrame(tSNE_tweets_with_scores, columns = ['D1', 'D2', 'D3', 'Score'])

In [89]:
tsne_df

,D1,D2,D3,Score
0,1.746208,-10.592792,-9.030455,0.0
1,0.105036,-13.407832,0.306753,0.0
2,-7.674889,-9.956732,0.518595,0.0
3,-6.114563,7.945170,12.378880,0.0
4,8.282591,4.886459,14.835587,0.0
5,6.323470,-10.840482,9.946914,0.0
6,10.425945,9.308215,7.654966,0.0
7,-4.788845,1.735339,-13.159933,0.0
8,-1.409564,17.616214,7.117903,0.0
9,-10.838679,-10.383621,9.771331,0.0


In [91]:
trace0 = go.Scatter3d(
    x=tsne_df.query('Score == 0.0')['D1'],
    y=tsne_df.query('Score == 0.0')['D2'],
    z=tsne_df.query('Score == 0.0')['D3'],
    name = 'Score 0',
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)

trace1 = go.Scatter3d(
    x=tsne_df.query('Score == 1.0')['D1'],
    y=tsne_df.query('Score == 1.0')['D2'],
    z=tsne_df.query('Score == 1.0')['D3'],
    name = 'Score 1',
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)


data = [trace0, trace1]
layout = go.Layout(
    title='Differences between Positive and Negative Feelings for Question 1', 
    font=dict(size=16, color='#7f7f7f'),
    height = 800, 
    width = 900,
    legend=dict(
        x=0,
        y=0.90,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)', 
        font=dict(
            size=14,
            color='#000'
            ),
        ),
    )



fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Differences between Positive and Negative Feelings')

In [7]:
#Smaller Sample 
sample_tsne_df = tweet_data.sample(n=200)

In [9]:
#For kicks and giggles, I want to see how the neutral scores look like. 
c_vectorizer = CountVectorizer(decode_error= 'ignore', min_df = 20, ngram_range=(1,2) )
c_X = c_vectorizer.fit_transform(sample_tsne_df.Tweet).toarray()

In [10]:
sample_tsne_df['Sentiment_score'] = sample_tsne_df['Sentiment'].map(lambda x: 1 if x == 'positive' else -1 if x == 'negative' else 0)

In [16]:
tSNE_model = TSNE(n_components=3, random_state=42)
tSNE_tweets_all = tSNE_model.fit_transform(c_X)
tSNE_tweets_all_with_scores = np.column_stack((tSNE_tweets_all,sample_tsne_df['Sentiment_score'].values))
tsne_all_df = pd.DataFrame(tSNE_tweets_all_with_scores, columns = ['D1', 'D2', 'D3', 'Score'])

In [17]:
tsne_all_df

,D1,D2,D3,Score
0,-0.154956,37.858900,-17.116246,-1.0
1,7.629307,-1.950003,-16.401159,0.0
2,22.879979,24.368328,-0.405617,-1.0
3,22.639232,-4.441329,-8.808797,-1.0
4,12.808466,-31.422360,15.136137,-1.0
5,0.454303,-22.777795,24.999724,-1.0
6,2.860567,29.155608,-2.886472,-1.0
7,-19.407496,-25.456273,-6.110827,0.0
8,4.166576,1.705105,-36.925219,-1.0
9,-12.683949,-26.087312,4.496626,-1.0


In [20]:
#Very different due to adding an extra category (curse of dimensionality)

trace0 = go.Scatter3d(
    x=tsne_all_df.query('Score == 0.0')['D1'],
    y=tsne_all_df.query('Score == 0.0')['D2'],
    z=tsne_all_df.query('Score == 0.0')['D3'],
    name = 'Score 0',
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)

trace1 = go.Scatter3d(
    x=tsne_all_df.query('Score == 1.0')['D1'],
    y=tsne_all_df.query('Score == 1.0')['D2'],
    z=tsne_all_df.query('Score == 1.0')['D3'],
    name = 'Score 1',
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)

trace2 = go.Scatter3d(
    x=tsne_all_df.query('Score == -1.0')['D1'],
    y=tsne_all_df.query('Score == -1.0')['D2'],
    z=tsne_all_df.query('Score == -1.0')['D3'],
    name = 'Score -1',
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)


data = [trace0, trace1, trace2]
layout = go.Layout(
    title='Differences between Positive, Negative, and Neutral Feelings', 
    font=dict(size=16, color='#7f7f7f'),
    height = 800, 
    width = 900,
    legend=dict(
        x=0,
        y=0.90,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)', 
        font=dict(
            size=14,
            color='#000'
            ),
        ),
    )



fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Differences between Positive, Negative, and Neutral Feelings')